# Lab 3: Gaussian process regression

### Machine Learning 1, September 2015

* The lab exercises should be made in groups of two, three or four people.
* The deadline is October 25th (Sunday) 23:59.
* Assignment should be sent to Philip Versteeg (p.j.j.p.versteeg@uva.nl). The subject line of your email should be "lab\#\_lastname1\_lastname2\_lastname3".
* Put your and your teammates' names in the body of the email.
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file follows the same rule as the subject line. For example, if the subject line is "lab01\_Kingma\_Hu", the attached file should be "lab01\_Kingma\_Hu.ipynb". Only use underscores ("\_") to connect names, otherwise the files cannot be parsed.

Notes on implementation:

* You should write your code and answers in an IPython Notebook: http://ipython.org/notebook.html. If you have problems, please contact us.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* NOTE: Make sure we can run your notebook / scripts!
$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bxp}{\mathbf{x}^{'}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\ba}{\mathbf{a}}$
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bv}{\mathbf{v}}$
$\newcommand{\bq}{\mathbf{q}}$
$\newcommand{\bp}{\mathbf{p}}$
$\newcommand{\bh}{\mathbf{h}}$
$\newcommand{\bI}{\mathbf{I}}$
$\newcommand{\bX}{\mathbf{X}}$
$\newcommand{\bT}{\mathbf{T}}$
$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bV}{\mathbf{V}}$
$\newcommand{\xm}{\mathbf{x}_m}$
$\newcommand{\xn}{\mathbf{x}_n}$
$\newcommand{\y}{\mathbf{y}}$
$\newcommand{\K}{\mathbf{K}}$
$\newcommand{\zero}{\mathbf{0}}$
$\newcommand{\yi}{\y_i}$
$\newcommand{\thetav}{\mathbf{\theta}}$
$\newcommand{\t}{\mathbf{t}}$
$\newcommand{\x}{\mathbf{x}}$
$\newcommand{\tN}{\mathbf{t}_N}$
$\newcommand{\xN}{\mathbf{x}_N}$
$\newcommand{\k}{\mathbf{k}}$
$\newcommand{\C}{\mathbf{C}}$
$\newcommand{\CN}{\mathbf{C}_N}$
$\newcommand{\KN}{\mathbf{K}_N}$
$\newcommand{\eyeN}{\mathbf{I}_N}$

# Gaussian process regression

For this Lab we will be refer to Bishop sections 6.4.2 and 6.4.3. You may also want to refer to Rasmussen's Gaussian Process text which is available online at http://www.gaussianprocess.org/gpml/chapters/ and especially to the project found at http://www.automaticstatistician.com/index.php by Ghahramani for some intuition in GP.  To understand Gaussian processes, it is highly recommended understand how marginal, partitioned Gaussian distributions can be converted into conditional Gaussian distributions.  This is covered in Bishop 2.3 and summarized in Eqns 2.94-2.98.




### Sinusoidal Data

We will use the same data generating function that we used previously for regression.  You can change sigma/beta, but keep it reasonable.  Definitely play around once you have things working.  Make use of these functions as you wish.

In [ ]:
%pylab inline
import pylab as pp

In [ ]:
sigma = 0.5
beta  = sigma ** (-2) # this is the beta used in Bishop Eqn. 6.59
N_test = 100
x_test = np.linspace(-1, 1, N_test); 
mu_test = np.zeros(N_test)

In [ ]:
def true_mean_function( x ):
    # return np.sin( 2*pi*(x+1) ) 
    # The +1 is pointless, sin(x) = sin(x+2pi)
    return np.sin( 2*pi*x)
    
def add_noise( y, sigma ):
    # Noise is drawn from Normal(0, sigma)
    noise = 0 + sigma*np.random.randn(len(y))
    return y + noise 

def generate_t( x, sigma ):
    # This corresponds to Bishop (6.57)
    # This is simply sinusoidal data with gaussian noise
    return add_noise( true_mean_function(x), sigma )

In [ ]:
y_test = true_mean_function( x_test ) # 'True' function values
t_test = add_noise( y_test, sigma )   # Target values with noise
pp.plot( x_test, y_test, 'b-', lw=2)
pp.plot( x_test, t_test, 'go')

### 1. Sampling from the Gaussian process prior (30 points)

We will implement Gaussian process regression using the kernel function in Bishop Eqn. 6.63.  

#### 1.1 k_n_m( xn, xm, thetas ) (10 points)
To start, implement function "k_n_m( xn, xm, thetas )" that takes scalars $\xn$ and $\xm$, and a vector of $4$ thetas, and computes the kernel function Bishop Eqn. 6.63 (10 points). 

**NOTE:** We deal with one dimensional data. That's why $x_n$ and $x_m$ are scalars and not vectors. So whereas in Bishop, the kernel matrix is ${\bf K} = [ k({\bf x}_n, {\bf x}_m) ]_{n,m\le N}$, here the ${\bf x_n}$ are just numbers. Hence

\begin{align}\tag{6.63'}
{\bf K}_{nm} = k(x_n, x_m) = \theta_0 \exp\Bigl( \frac{-\theta_1}{2} |x_n - x_m|^2\Bigr) + \theta_2 + \theta_3x_nx_m
\end{align}


In [ ]:
def k_n_m(xn, xm, thetas):
    return thetas[0] * np.exp((-thetas[1]) / 2.0 * (xn - xm)**2) + thetas[2] + thetas[3] * xn * xm

#### 1.2 computeK( X1, X2, thetas ) (5 points)
Eqn 6.60 is the marginal distribution of mean ouput of $N$ data vectors: $p(\y) = \mathcal{N}(\zero, \K)$.  Notice that the expected mean function is $0$ at all locations, and that the covariance is a $N$ by $N$ kernel matrix $\K$.  Write a function "computeK( X1, X2, thetas )" that computes the kernel matrix. Hint: use k_n_m as part of an innner loop (of course, there are more efficient ways of computing the kernel function making better use of vectorization, but that is not necessary) (5 points).  

**NOTE.** Here, I don't see why K should be computed using some $X_1$ and $X_2$. I suppose the capital $X$ suggests that we are dealing with vectors $X_1$ and $X_2$. Or, more precisely, it suggest that we are dealing with an $N \times D = N\times 1$ matrix of all datapoints: $X_1 = (x_1, \dots, x_N)^T$. Indeed, we kan calculate $\bf K$ from that. But for that, we don't need an $X_2$ of..., yeah, of what, really?

In [ ]:
def computeK( X1, X2, thetas):
    K = zeros([len(X1), len(X2)])
    for i in range(len(X1)):
        for j in range(len(X2)):
            K[i,j] = k_n_m( X1[i], X2[j], thetas)
    return K

#### 1.3 Plot function samples (15 points)
Now sample mean functions at the x_test locations for the theta values in Bishop Figure 6.5, make a figure with a 2 by 3 subplot and make sure the title reflects the theta values (make sure everything is legible).  In other words, sample $\yi \sim \mathcal{N}(\zero, \K_{\thetav})$.  Make use of numpy.random.multivariate_normal().  On your plots include the expected value of $\y$ with a dashed line and fill_between 2 standard deviations of the uncertainty due to $\K$ (the diagonal of $\K$ is the variance of the model uncertainty) (15 points).

**NOTE.** On the diagonal of $\bf K$ we find values of the form $k(x,x)$. Now
\begin{align}
k(x,x) = \theta_0 + \theta_2 + \theta_3 x^2.
\end{align}
Hence 
\begin{align}
\text{diag}({\bf K}) = \begin{cases}
    (\theta_0 + \theta_2) {\bf I}_N &\text{if $\theta_3 = 0$}\\
    (\theta_0 + \theta_2)  \text{diag}(x_1^2, \dots, x_N^2) &\text{if $\theta_3 \neq 0$}
\end{cases}
\end{align}
As the standard deviation is given by $2 \sqrt{\text{diag}({\bf K})}$, it follows that the standard deviation is constant and equal to $\theta_0 + \theta_2$ whenever $\theta_3=0$ and varies otherwise.


In [ ]:
thetas_list = [[[1, 4, 0, 0],    [9, 4, 0, 0],  [1, 64, 0, 0]], 
          [[1, 0.25, 0, 0], [1, 4, 10, 0], [1, 4, 0, 5]]]
num_samples = 5
f, axarr = subplots(2,3, figsize=(12,8))
f.tight_layout()

for i in range(2):
    for j in range(3):
        maxes, mins = [], [];
        
        # Covariance matrix
        thetas = thetas_list[i][j]
        K = computeK(x_test, x_test, thetas)
        std_total = np.sqrt(np.diag(K))
        mins += [min(-2 * std_total)]
        maxes += [max(2 * std_total)]
        
        # Expectation of y
        axarr[i,j].plot(zeros(N_test), 'k--')

        axarr[i,j].fill_between( range(N_test), 2*std_total, -2*std_total, color='r', alpha=0.15 )
        title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$"
        axarr[i,j].set_title(title % tuple(thetas))

        for k in range(num_samples):
            y = np.random.multivariate_normal( zeros(N_test), K)
            maxes += [max(y)]
            mins += [min(y)]
            axarr[i,j].plot(y)
        
        m = min(mins); M=max(maxes);
        axarr[i,j].set_ylim([m - (M-m)*.1, M+ (M-m)*.1])


### 2. Predictive distribution (35 points)

So far we have sampled mean functions from the prior.  We can draw actual data $\t$ two ways.  The first way is generatively, by first sampling $\y | \K$, then sampling $\t | \y, \beta$ (Eqns 6.60 followed by 6.59).  The second way is to integrate over $\y$ (the mean draw) and directly sample $\t | \K, \beta$ using Eqn 6.61.    This is the generative process for $\t$.  Note that we have not specified a distribution over inputs $\x$;  this is because Gaussian processes are conditional models.  Because of this we are free to generate locations $\x$ when playing around with the GP; obviously a dataset will give us input-output pairs.

Once we have data, we are interested in the predictive distribution (note: the prior is the predictive distribution when there is no data).  Consider the joint distribution for $N+1$ targets, given by Eqn 6.64.  Its covariance matrix is composed of block components $\CN$, $\k$, and $c$.  The covariance matrix $CN$ for $\tN$ is $\CN = \KN + \eyeN / \beta$.  We have just made explicit the size $N$ of the matrix; $N$ is the number of training points.  The kernel vector $\k$ is a $N$ by $1$ vector of kernel function evaluations between the training input data and the test input vector.  The scalar $c$ is a kernel evaluation at the test input.

#### 2.1 gp_predictive_distribution(...) (10 points)
Write a function "gp_predictive_distribution(x_train, t_train, x_test, theta, beta, C = None)" that computes  Eqns 6.66 and 6.67, except allow for an arbitrary number of test points (not just one) and now the kernel matrix is for training data.  By having C as an optional parameter, we can avoid computing it more than once (for this problem it is unimportant, but for real problems this is an issue).  The function should compute $\C$, $\k$, and $c$ and the mean and noise functions.  Do not forget: the computeK function computes $\K$, not $\C$! (10 points)

In [ ]:
def gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, C = None, invC = None):
    
    # Dimensions
    N, M = len(x_train), len(x_test)
    
    if C == None:
        # Covariance matrix from Bishop's (6.62)
        C = computeK(x_train, x_train, thetas) + np.eye(N)/beta
        
    if invC == None:
        invC = np.linalg.inv(C)
    
    # We now calculate the new C; C_test. Roughly, that is C_{N+1} 
    # in (6.65), but now for multiple new test points. 
    C_test = np.zeros([N + M, N + M])
    
    # Top left part: just C
    C_test[0:N, 0:N] = C
    
    # Bottom right part
    c = computeK(x_test, x_test, thetas) + np.eye(M)/beta
    C_test[N:,N:] = c

    # Top right and bottom left:
    k = computeK(x_train, x_test, thetas)
    C_test[0:N, N:] = k
    C_test[N:, 0:N] = k.T
    
    # Parameters of distribution
    _A = k.T.dot(invC)
    mu_test  = _A.dot(t_train)   # (6.66)
    var_test = c - _A.dot(k)     # (6.67)
    
    return C, invC, mu_test, var_test

#### 2.2 gp_log_likelihood(...) (10 points)
Later, to learn the hyperparameters, we will need to compute the log-likelihood of the of the training data.  Implicitly, this is conditioned on the value setting for $\thetav$.  Write a function "gp_log_likelihood( x_train, t_train, theta, C = None, invC = None )", where C and invC can be stored and reused.  (10 points)

**NOTE.** The questions suggests we can pass $\bf C$ and ${\bf C}^{-1}$ as *optional* arguments to the function. However, none of those can be calculated without $\beta$, which is not an argument of `gp_log_likelihood`. Therefore, we don't consider the arguments $\bf C$ and ${\bf C}^{-1}$ to be optional. Also, it follows that the argument `thetas` is redundant.

In [ ]:
def gp_log_likelihood( x_train, t_train, C, invC ):
    # The log likelihood is given in Bishop's (6.69):
    return -0.5 * ( log(det(C)) + t_train.dot( invC.dot(t_train) ) + len(x_train) * log(2*pi))

#### 2.3 Plotting (10 points)
Repeat the 6 plots above, but this time conditioned on the training points.  Use the sinuosoidal data generator to create 2 training points where x is sampled uniformly between $-1$ and $1$.  For these plots, feel free to use the provided function "gp_plot".  Make sure you put the parameters in the title and this time also the log-likelihood. (10 points)  Try to understand the two types of uncertainty!  If you do not use "gp_plot", please add a fill between for the model and target noise. 

In [ ]:
def gp_plot( ax, x_test, y_test, mu_test, var_test, x_train, t_train, theta, beta ):
    # x_test:   the test data
    # y_test:   the true function at x_test
    # mu_test:  predictive mean at x_test
    # var_test: predictive covariance at x_test 
    # t_train:  the training values
    # theta:    the kernel parameters
    # beta:     the precision (known)
    
    # the reason for the manipulation is to allow plots separating model and data stddevs.
    std_total = np.sqrt(np.diag(var_test))         # includes all uncertainty, model and target noise 
    std_model = np.sqrt( std_total**2 - 1.0/beta ) # remove data noise to get model uncertainty in stddev
    std_combo = std_model + np.sqrt( 1.0/beta )    # add stddev (note: not the same as full)
    
    ax.plot( x_test, y_test, 'k', lw=2)
    ax.plot( x_test, mu_test, 'k--')
    ax.fill_between( x_test, mu_test+2*std_combo,mu_test-2*std_combo, color='k', alpha=0.1 )
    ax.fill_between( x_test, mu_test+2*std_model,mu_test-2*std_model, color='r', alpha=0.25 )
    ax.plot( x_train, t_train, 'ro', ms=7 )

In [ ]:
def plotstuff( x_train, t_train ):
    
    thetas_list = [[[1, 4, 0, 0],    [9, 4, 0, 0],  [1, 64, 0, 0]], 
                  [[1, 0.25, 0, 0], [1, 4, 10, 0], [1, 4, 0, 5]]]
    f, axarr = subplots(2,3, figsize=(12,8))

    for i in range(2):
        for j in range(3):
            
            thetas = thetas_list[i][j]
            C, invC, mu_test, var_test = gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, None, None)
            ln_p = gp_log_likelihood( x_train, t_train, C, invC)
            
            # Plot everything
            gp_plot( axarr[i,j], x_test, y_test, mu_test, var_test, x_train, t_train, thetas, beta)
            title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$ and $\\ln(p) =%.1f$"
            axarr[i,j].set_title(title % tuple(thetas + [ln_p]))
    

# Training data and plotting
x_train = np.random.rand(2)*2 - 1
t_train = generate_t( x_train, sigma )
plotstuff(x_train, t_train)

#### 2.4 More ploting (5 points)
Repeat the 6 plots above, but this time conditioned a new set of 10 training points. (5 points)

In [ ]:
x_train = np.random.rand(10)*2-1
t_train = generate_t( x_train, sigma )
plotstuff(x_train, t_train)

### 3. Learning the hyperparameters (45 points)

Learning the values of the parameter $\thetav$ can be very tricky for Gaussian processes in general, but when the data is univariate like ours, we can visualize the fit and see how plausible it looks.

#### 3.1 Derivatives (5 points)
Maximum likelihood or MAP learning is the most common way of setting the parameters, though a fully Bayesian approach is possible too.  We will look at ML today.  For this, we start with the dervivative of the log-likelihood with respect to the parameters $\thetav$; this is Eqn 6.70.  This, in turn, requires the derivative of the kernel matrix $\CN$ wrt $\thetav$.  This is the matrix of element-wise derivatives of the kernel function.  Write the derivatives for $\theta_0$ to $\theta_3$ for our kernel function (5 points).  

$\partial_{{\bf \theta}_0} K = \left[ \exp\left\{ -{\theta_1 \over 2} \|x_n - x_m\|^2 \right\}\right]_{0\leq n,m \leq N}$

$\partial_{{\bf \theta}_1} K = \left[ -{{\bf \theta}_0 \over 2} \|x_n - x_m\|^2 \exp\left\{ - {{\bf \theta}_1 \over 2} \|x_n - x_m\|^2 \right\}\right]_{0\leq n,m \leq N}$

$\partial_{{\bf \theta}_2} K = \left[ 1\right]_{0\leq n,m \leq N}$

$\partial_{{\bf \theta}_3} K = \left[ x_n^\top x_m\right]_{0\leq n,m \leq N}$


#### 3.2 Questions (5 points)
Which parameters in $\thetav$ are unconstrained, that is, where any positive/ negative values are valid? (5 points)

The only possible constraint we see is that 
\begin{align}\tag{6.63'}
    k(x_n, x_m) = \theta_0 \exp\Bigl( \frac{-\theta_1}{2} |x_n - x_m|^2\Bigr) + \theta_2 + \theta_3x_nx_m
\end{align}
should be a valid kernel.

** The incomplete approach.**
The straightforward — but incomplete — approach would be the following: (6.63') is a kernel if the three summands are kernels, by (6.17). The first summand, by (6.13), is a kernel if $\theta_0 > 0$ and $\exp\Bigl( \frac{-\theta_1}{2} |x_n - x_m|^2\Bigr)$ is a kernel, which by (6.23) is the case if $\theta_1 >0$. The second summand, $\theta_2$, is a kernel when it can be seen as a 0-th order polymial with *nonnegative* coefficients, that is, when $\theta_2\ge0$. Finally, the last summand is a kernel when $\theta_3 >0$, again by (6.13). Summarizing, we find that $k$ is a valid kernel whenever $\theta_0, \theta_1,\theta_3 >0$ and $\theta_2 \ge 0$.

True as that may be, this argument does not show that for other values of $\theta_i$, $k$ will *not* be a kernel. So we have sufficient conditions, but not the necessary conditions for $k$ to be a kernel.
And indeed, these conditions are not necessary: we can clearly take $\theta_0 = \theta_1 = \theta_2 = 0$ and then $k$ will still be a valid kernel, if in any case $\theta_3 >0$.

** A completer approach. **
To get necessary conditions, we need to know what exactly qualifies as a kernel function. The clearest characterization in Bishop is that the Gram matrix ${\bf K} = [k(x_n, x_m)]$ should be positive semidefinite for all possible choices $\{x_1, \dots, x_n\}$. Deriving conditions on the $\theta$'s from this characterization, is however highly nontrivial and we are definitely not going to try to do so.

#### 3.3 More derivatives (5 points)
For parameters that are constrained to be positive, the usual approach is to use the exponential of the free-parameter in the kernel function, but perform gradient ascent on the unconstrained values.  Consider the case  $\theta_i = \exp( \phi_i)$, where $\phi_i$ is unconstrained.  Write the derivative for $\phi_i$ in terms of the derivatives you already computed (5 points).  Hint: use the chain rule and do not repeat the full derivation.


$\partial_{\phi_i} K = \partial_{\phi_i} \thetav_i \partial_{\thetav_i} K = \exp(\phi_i) \partial_{\thetav_i} K$

#### 3.4 Grid search (10 points)
Grid-search: for the same training set you have above, perform a small grid search over $\thetav$ (try at least 20 combinations).  Have your grid-search loop or function print out rows of log-likelihood + $\thetav$ sorted by best to worst.  Use the log-likelihood to select the best $\thetav$ and the worst.  Plots both the same way as the subplots above (ie a 1 by 2 subplot of best and worst). (10 points)

In [ ]:
thetas_list = zeros((81,4))
ln_p_list = zeros(81)
mu_test_list = zeros((81,100))
var_test_list = zeros((81,100,100))
i = 0

for theta0 in [1,5,10]:
    for theta1 in [1,10,50]:
        for theta2 in [0,5,10]:
            for theta3 in [0,5,10]:
                thetas = [theta0, theta1, theta2, theta3]
                thetas_list[i] = thetas
                C, invC, mu_test_list[i], var_test_list[i] = gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, None, None)
                ln_p = gp_log_likelihood( x_train, t_train, C, invC)
                ln_p_list[i] = ln_p
                i += 1

best2worst = np.argsort(ln_p_list)[::-1]
thetas_sorted = thetas_list[best2worst]
ln_p_sorted = ln_p_list[best2worst]
mu_test_sorted = mu_test_list[best2worst]
var_test_sorted = var_test_list[best2worst]
for i in range(81):
    print(str(ln_p_sorted[i])+"\t"+str(thetas_sorted[i]))

  
f,axarr = subplots(1,2, figsize=(8,4))
gp_plot( axarr[0], x_test, y_test, mu_test_sorted[0], var_test_sorted[0], x_train, t_train, thetas_sorted[0], beta)
title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$ and $\\ln(p) =%.1f$"
axarr[0].set_title(title % (thetas_sorted[0][0], thetas_sorted[0][1], thetas_sorted[0][2], thetas_sorted[0][3], ln_p_sorted[0]))

gp_plot( axarr[1], x_test, y_test, mu_test_sorted[-1], var_test_sorted[-1], x_train, t_train, thetas_sorted[-1], beta)
title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$ and $\\ln(p) =%.1f$"
axarr[1].set_title(title % (thetas_sorted[-1][0], thetas_sorted[-1][1], thetas_sorted[-1][2], thetas_sorted[-1][3], ln_p_sorted[-1]))

#### 3.5 Questions (10 points)
Selecting kernel functions can be somewhat of an art.  There are charateristics of kernel functions that are useful for some data sets, but not others.  Complicating the matter is the ability to combine kernels with different characteristics (long term trends + seasonal fluctuations).  Describe the charactistics of the kernel function we are using in terms of (signal, scale, offsets, etc). You may want to play around with $\thetav$ and see what each parameter does/affects/etc.  (5 points)  Describe why the best parameters work well for the training data and explain why the bad parameter settings perform poorly (in terms of the first part of the question).  (5 points)

### Introduction
We look at the effects of different parameter values for the predictive distribution, both without data (i.e. the prior) and with data.

In [ ]:
def plot_for_thetas(thetas_list, ylim=None):
    num_samples = 5
    f, axarr = subplots(1, len(thetas_list), figsize=(4*len(thetas_list),4))
    f.tight_layout()
    x_test = np.linspace(-1, 1, 100); 
    for i, thetas in enumerate(thetas_list):
            maxes, mins = [], [];

            # Covariance matrix
            K = computeK(x_test, x_test, thetas)
            std_total = np.sqrt(np.diag(K))
            mins += [min(-2 * std_total)]
            maxes += [max(2 * std_total)]

            # Expectation of y
            axarr[i].plot(x_test, zeros(len(x_test)), 'k--')

            axarr[i].fill_between(x_test, 2*std_total, -2*std_total, color='r', alpha=0.15 )
            title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$"
            axarr[i].set_title(title % tuple(thetas))

            for k in range(num_samples):
                numpy.random.seed(k)
                y = np.random.multivariate_normal( zeros(N_test), K)
                maxes += [max(y)]
                mins += [min(y)]
                axarr[i].plot(x_test, y)

            m = min(mins); M=max(maxes);
            if ylim != None: 
                axarr[i].set_ylim(ylim)
            else:
                axarr[i].set_ylim([m - (M-m)*.1, M+ (M-m)*.1])  

In [ ]:
def plot_for_thetas_data( thetas_list, ylim=None ):
    f, axarr = subplots(1, len(thetas_list), figsize=(4*len(thetas_list),4))

    for i, thetas in enumerate(thetas_list):
        C, invC, mu_test, var_test = gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, None, None)
        ln_p = gp_log_likelihood( x_train, t_train, C, invC)

        # Plot everything
        gp_plot( axarr[i], x_test, y_test, mu_test, var_test, x_train, t_train, thetas, beta)
        title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$"
        axarr[i].set_title(title % tuple(thetas))
        if ylim != None:
            axarr[i].set_ylim(ylim)  

In [ ]:
# Training data and plotting
x_train = np.random.rand(20)*2 - 1
t_train = generate_t( x_train, sigma )

### The effect of $\theta_0$

In the plots below, we increased $\theta_0$ exponentially. It is clearly visible how the $y$-axis scales along (in some, roughly logarithmic, way).

A multivariate Gaussian distribution with covariance matrix $K$ has the following interpretation: the diagonal of $K$ determines the variance of the data points, while the off-diagonal determines covariance between points. High covariance implies high correlation; in other words, large off-diagonal values imply that values will behave comparably.

Increasing $\theta_0$ increases the diagonal elements in a linear fashion; off-diagonal elements are increased with less speed. Therefore we see in the first plot that the red bar increases exponentially.

In [ ]:
plot_for_thetas([[t, 1, 0, 0] for t in exp([-99,0, 2, 3, 4])], [-16,16])

In [ ]:
plot_for_thetas_data([[t, 1, 0, 0] for t in exp([-99,0, 4, 8, 12])], [-6,6])

### The effect of $\theta_1$
Below are several plots for $\theta_1 \in \{e^0, e^2, e^4, e^6\}$. Clearly, the variability of the functions increases as $\theta_1$ increases. This is again explained by the interpretation of $K$ as the covariance matrix; increasing $\theta_1$ has no effect on the diagonal, hence the variance is constant. But the covariance becomes lower for larger $\theta_1$'s, so we see highly irregular/oscillatory functions in the rightmost graph. By equation (6.59), this irregularity is propagated forward to the target predictive distribution.

In [ ]:
plot_for_thetas([[1, t, 0, 0] for t in exp([-99,0, 2, 4, 6])], [-3,3])

As we are fitting sinusoidal data, this is a very important parameter:

In [ ]:
plot_for_thetas_data([[1, t, 0, 0] for t in exp([-99,0, 2, 4, 6])], [-3,3])

### The effect of $\theta_2$
In the plots below, the scale is identical to the plots above (for $\theta_0$). The difference with the plots above is that there is no scaling, but rather the **relative offset** increases as $\theta_2$ increases.

Looking a tthe covariance interpretation, we see that $\theta_2$ is added to $K$ at every position: therefore the variance increases as quickly as the covariances do.

In [ ]:
plot_for_thetas([[1, 1, t, 0] for t in  exp([0,2,3,4])], [-16,16])

We see that qualitatively, the distributions are very similar. We see that in the predictive distribution, this similarity is very clear; $\theta_2$ has hardly any effects:

In [ ]:
plot_for_thetas_data([[1, 1, t, 0] for t in exp([0,4,8,12])], [-4,4])

### The effect of $\theta_3$
We already investigated the effect of $\theta_3$ above. There we noted that when $\theta_3=0$, the variance is constant. However, when we increase $\theta_3$, the variance between functions increase. Around 0, however, the variation between function is constant, which should make sense since the distribution is centered around zero.

In [ ]:
plot_for_thetas([[1, 1, 1, t] for t in [0,4,16,64]], [-15,15])

Even though the above graph shows clear qualitative differences, the predictive distribution looks exactly the same:

In [ ]:
plot_for_thetas_data([[1, 1, 1, t] for t in [4,100,1000]], [-2,2])

### Conclusion

We see that many of the characteristics of the $y$-distribution are explained by looking at the Gram matrix $K$ and its interpretation in covariance sense. However, this does not explain all characteristics of the predictive distribution.

One thing we can definitely make up is that $\theta_1$ acts more or less as a regularisation parameter: the bias-variance tradeoff is very obvious in the data plot of the $\theta_1$ inspection. Knowing that the optimal value of a regularisation parameter usually involves trial-and-error, we conclude that choosing $\theta_1$

#### 3.6 Bonus: Implementation (20 points)
Implement gradient-ascent (or descent if you wish) using the combination of a) the log-likelihood objective function and b) the gradients you calculated above.  Run on the training data above and show the log-likehood curve as it learns and a plot of the final model.  Feel free to use available software (eg search for "minimize.py" which uses conjugate gradient descent, or something in scipy).  NB: log-likelihood should be monotonically increasing.  You are encouraged to also search and use "checkgrad".  (20 points)

In [ ]:
from scipy.optimize import minimize, check_grad

def neg_log_likelihood_for_phis(phis):
    thetas = exp(phis)
    C, invC, mu_test, var_test = gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, None, None)
    ln_p = gp_log_likelihood( x_train, t_train, C, invC)
    return -ln_p

def jacobian(phis):
    N = len(x_train)
    thetas = exp(phis)
    C = computeK(x_train, x_train, thetas) + np.eye(N)/beta
    invC = np.linalg.inv(C)
    # since C minus K is constant, the partial derivatives of C are the same as those of K we calculated in 3.1
    parder_C_0 = zeros((N,N))
    parder_C_1 = zeros((N,N))
    parder_C_2 = zeros((N,N))
    parder_C_3 = zeros((N,N))
    for n in range(N):
        for m in range(N):
            parder_C_0[n,m] = -exp(phis[0])*np.exp(-thetas[1]/2*(x_train[n]-x_train[m])**2)
            parder_C_1[n,m] = -exp(phis[1])*thetas[0]/2*(x_train[n]-x_train[m])**2*parder_C_0[n,m]
            parder_C_2[n,m] = -exp(phis[2])
            parder_C_3[n,m] = -exp(phis[3])*np.dot(x_train[n].T,x_train[m])
    par_der0 = -0.5*np.trace(invC.dot(parder_C_0))+0.5*t_train.T.dot(invC.dot(parder_C_0.dot(invC.dot(t_train))))
    par_der1 = -0.5*np.trace(invC.dot(parder_C_1))+0.5*t_train.T.dot(invC.dot(parder_C_1.dot(invC.dot(t_train))))
    par_der2 = -0.5*np.trace(invC.dot(parder_C_2))+0.5*t_train.T.dot(invC.dot(parder_C_2.dot(invC.dot(t_train))))
    par_der3 = -0.5*np.trace(invC.dot(parder_C_3))+0.5*t_train.T.dot(invC.dot(parder_C_3.dot(invC.dot(t_train))))
    return np.array([par_der0,par_der1,par_der2,par_der3])

ln_p_list = []
def collect_ln_p(phis,ln_p_list):
    ln_p = -neg_log_likelihood_for_phis(phis)
    ln_p_list.append(ln_p)
    return ln_p_list

x0 = [0,10,-100,-100]
print "Difference between calculated gradient and Python's estimation:"
print check_grad(neg_log_likelihood_for_phis,jacobian,x0)

res = minimize(neg_log_likelihood_for_phis, x0, method='cg', jac=jacobian, callback=lambda phis: collect_ln_p(phis,ln_p_list))
print "log-likehood curve as it learns:"
pp.plot(range(len(ln_p_list)),ln_p_list)
pp.show()

thetas = np.exp(res.x)
print "Plot for the optimized model:"
f, axarr = subplots(1,1)
C, invC, mu_test, var_test = gp_predictive_distribution( x_train, t_train, x_test, thetas, beta, None, None)
ln_p = gp_log_likelihood( x_train, t_train, C, invC)
gp_plot( axarr, x_test, y_test, mu_test, var_test, x_train, t_train, thetas, beta)
title = "$\\mathbf{\\theta} = (%.1f,\; %.1f,\; %.1f,\; %.1f)$ and $\\ln(p) = %.1f$"
axarr.set_title(title % (thetas[0], thetas[1], thetas[2], thetas[3], ln_p))